# 🎨 Stable Diffusion XL Image Generator
Generate high-quality images with Stable Diffusion XL (SDXL) 1.0 using Colab’s free GPU.

In [ ]:
# ─── Setup ────────────────────────────────────────────────────────────
# 1) Clear pip cache
!pip cache purge

# 2) Install all compatible versions at once:
!pip install -q \
    diffusers==0.29.1 \
    transformers>=4.41.0,<5.0.0 \
    accelerate==0.29.0 \
    huggingface_hub>=0.25.0 \
    safetensors==0.4.2 \
    gradio==4.29.0 \
    torch \
    bitsandbytes \
    xformers \
    websockets>=13.0,<15.0dev

## Import Dependencies

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from transformers import BitsAndBytesConfig
import gradio as gr
from PIL import Image

## Model Loading Helpers

In [ ]:
# 8-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
)
pipe = None
refiner = None

def load_base_model():
    global pipe
    if pipe is None:
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config,
        )
        try:
            pipe.enable_xformers_memory_efficient_attention()
        except:
            pass

def load_refiner_model():
    global refiner
    if refiner is None:
        refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-refiner-1.0",
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config,
        )
        try:
            refiner.enable_xformers_memory_efficient_attention()
        except:
            pass

## Generation Function

In [ ]:
def generate_image(
    prompt:str,
    negative_prompt:str="",
    num_steps:int=30,
    guidance_scale:float=7.5,
    use_refiner:bool=False,
    refiner_steps:int=10,
    lora_model:str="",
    lora_scale:float=0.5,
    advanced_mode:bool=False
):
    load_base_model()
    kwargs = {}
    if advanced_mode and lora_model.strip():
        try:
            pipe.unet.load_attn_procs(lora_model)
            kwargs={"scale":lora_scale}
        except:
            return Image.new("RGB",(1024,1024),"black")

    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        cross_attention_kwargs=kwargs
    ).images[0]

    if advanced_mode and use_refiner:
        load_refiner_model()
        img = refiner(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=refiner_steps,
            image=img
        ).images[0]

    return img

## Gradio Interface

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🎨 SDXL Image Generator")
    with gr.Row():
        with gr.Column(scale=2):
            prompt = gr.Textbox(label="Prompt", lines=2)
            neg    = gr.Textbox(label="Negative Prompt", lines=1)
            steps  = gr.Slider(1,100,value=30,label="Steps")
            scale  = gr.Slider(1.0,20.0,value=7.5,label="Guidance")
            adv    = gr.Checkbox(label="Advanced Mode")
            ref    = gr.Checkbox(label="Use Refiner")
            rsteps = gr.Slider(1,50,value=10,label="Refiner Steps")
            lmodel = gr.Textbox(label="LoRA ID")
            lscale = gr.Slider(0.0,2.0,value=0.5,label="LoRA Scale")
            adv.change(lambda v: gr.update(visible=v),adv,[ref,rsteps,lmodel,lscale])
            btn = gr.Button("Generate")
        with gr.Column(scale=2):
            out = gr.Image(type="pil")

    btn.click(
        fn=generate_image,
        inputs=[prompt,neg,steps,scale,ref,rsteps,lmodel,lscale,adv],
        outputs=out
    )

    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=0,
        prevent_thread_lock=True
    )
